## Import packages

In [1]:
#image analysis
import skimage.io
import imageio
import alienlab.plot
from alienlab.improcessing import normalize, grey_to_rgb, make_binary
import alienlab.segment
from alienlab.fo import FramesOperator
import alienlab.io
from scipy import optimize

from alienlab.regression_func import *

#interactive widget packages
from ipywidgets import interact, interactive, fixed, interact_manual
from tkinter.filedialog import askopenfilename, askdirectory


import time
import os
import numpy as np
import matplotlib.pyplot as plt


%matplotlib qt

## Import video file

#### USER ACTION:

#### **Drag and drop a video file (.tif format) in the file section**

In [2]:
#file_path = "ENTER NAME OF THE FILE YOU DROPPED IN THE FILE SECTION HERE"
file_folder = askdirectory(title = 'Select an experiment folder') # pops up a window to select your file
# uncomment this line if you use this jupyter notebook locally
#

In [3]:
show = True #option to output intermediary images in the segmentation process

# Import video file in HQ and select ROI
file_path = file_folder + "/video.tiff"
direc = os.path.split(file_path)[0]


# Initialize plotting tools
g = alienlab.plot.ShowFigure()
g.figsize = (15,7)
g.save_folder = "images"
g.date = False
p = alienlab.plot.PlotFigure()
p.figsize = (15,7)
p.save_folder = "images"
p.date = False

## Pre-process the video

In [4]:
# read the stacked frame. dim = NxHxW (N images in the video, Heigt, Width)

frames_full = skimage.io.imread(file_path)

#frames_full = np.stack([frames_full[:,:,1]]*10, 0) 
#uncomment this line if you have a single RGB image. The [:,:,1] stands for selection of the green channel

FO = FramesOperator(frames_full)
im = normalize(FO.frames[0], 0, 1)
im = grey_to_rgb(im)*255

# CROP
#y, x = alienlab.io.select_roi(np.uint8(im)) #select area of interest
#FO.x = x
#FO.y = y
#FO.crop() #crop image

start_time = time.time()
FO.compute_stats() #compute various statistical values on the frames and the pixels
FO.normalize(0, 1)
print("--- Computed frames statistics in %04f seconds ---" % (time.time() - start_time))

#FO.global_stats: each array has size N, number of frames and represents the stats of each frame
#FO.frames_stats: each array has size FO.x, FO.y and is an image representing the N frames stats overlayed

if show:
    p.title = 'statistics'
    p.xlabel = 'frame number'
    p.ylabel = 'amplitude'
    p.label_list = ['max', 'min', 'mean', 'std']
    fig = p.plotting(np.asarray(FO.inds), [FO.global_stats['max'], 
                        FO.global_stats['min'], 
                        FO.global_stats['mean']])
    p.save_name = 'frames_stats'
    p.saving(fig)

''' IMAGE SEGMENTATION '''

# selection of the frames with high dynamics that will be used for the image segmentation process.
# Let M be the highest value taken by a pixel in all the frames of the video. The frame F is kept for processing only if at
# least one pixel in the frame F has a value above 0.8*M. 
FO.selected_inds = FO.select_frames(FO.global_stats['max'], FO.global_stats['max'].max()*0.8)


--- Computed frames statistics in 0.370931 seconds ---


### Show reference image

In [5]:

plt.figure(figsize = (5, 5))
FO.selected_inds = FO.select_frames(FO.global_stats['max'], FO.global_stats['max'].max()*0.98) # Select only images with high intensity to increase contrast and lower computation time

plt.imshow(FO.frames[FO.selected_inds].sum(axis = 0), cmap = 'gray')


## Image segmentation

In [6]:
def segment_image(contrast, autolevel, dist_max, dist_seg, disk_size, max_contrast, interact = True, showit = show):
    
    start_time = time.time()
    FO.selected_inds = FO.select_frames(FO.global_stats['max'], FO.global_stats['max'].max()*0.98) # Select only images with high intensity to increase contrast and lower computation time

    #apply contrast filter to all frames
    frames_contrast = FO.apply(skimage.filters.rank.enhance_contrast,  selem = skimage.morphology.disk(contrast))
    #apply autolevel filter to all frames
    frames_autolevel = FO.apply(skimage.filters.rank.autolevel, selem = skimage.morphology.disk(autolevel))
    #sum the contrast images to get a reference grey-level contrast image
    frame_contrast = np.sum(frames_contrast, axis = 0)
    #sum the autolevel images to get a reference grey-level autolevel image
    frame_autolevel = np.sum(frames_autolevel, axis = 0)
    #obtain contrast mask from reference contrast image
    mask_contrast = make_binary(frame_contrast, soft_hard = 1)
    #otbain autolevel mask from reference autolevel image
    mask_autolevel =  make_binary(frame_autolevel, soft_hard = 1)
    #intersection of contrast aud autolevel masks
    mask_intersect = mask_contrast * mask_autolevel
    #clean the masks with a binary opening
    mask_intersect = skimage.morphology.binary_opening(mask_intersect, selem = skimage.morphology.disk(disk_size))
    #reference image of altitude for the watershed
    auto_contrast = normalize(mask_intersect * frame_autolevel)
    print("--- Computed binary mask in %04f seconds ---" % (time.time() - start_time))

    g.cmap = "inferno"
    if showit:
        g.figsize = (40,15)
        g.title_list =  'contrast', 'contrast threshold', 'mask intersect','autolevel', 'autolevel threshold','segmentation image'
        g.col_num = 3
        fig = g.multi([frame_contrast, mask_contrast, mask_intersect, 
                       frame_autolevel, mask_autolevel,  auto_contrast])
        g.save_name = 'Segmentation reference'
        g.saving(fig)

    start_time = time.time()
    ref = auto_contrast
    mask = mask_intersect
    #locate the local maxima
    local_maxi = alienlab.segment.local_maxima(auto_contrast, max_contrast, g,
                                                     ref_distance = dist_max, mask = mask, show = showit)
    #perform watershed segmentation
    watershed_im_mask = alienlab.segment.watershed(ref, mask, local_maxi,
                                                         g, ref_distance = dist_seg, show = False)
    segmented = watershed_im_mask
    print("--- Computed segmentation in %04f seconds ---" % (time.time() - start_time))

    if showit:
        alienlab.segment.show_segmentation(FO, segmented, g)
        
    if interact == False:
        return watershed_im_mask, FO


In [7]:
mask, FO = segment_image(contrast = 2, autolevel = 5, dist_max = True, dist_seg=True, disk_size = 2, max_contrast = 3, interact = False, showit= True)

--- Computed binary mask in 2.009687 seconds ---


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


--- Computed segmentation in 0.742059 seconds ---


In [8]:
g.cmap = "tab20"
fig = g.multi(mask)

In [9]:
# Collect item labels

# Item time trajectories with overlaps
# create a dictionnary with one entry for each item:
'''
{ '1.0': {'x_coords': np array, x coordinates in HQ}
            'y_coords': np array,  y coordinates in HQ
            'binned_coords': set, couples of (x,y) coordinates in binned video
            'surface': number of pixels in the item in HQ
            'pixel_values': array, size: (N, s) where N is number of frames and s surface
            'mean': array, size N, mean value of the item intensity for each frame
            'std':  array, size N, std value of the item intensity for each frame
            'remains' : True, the item is present in this segmentation step
            }
'2.0': {'x_coords'...
                }
    }
'''
segmented = mask
items = np.unique(segmented) #returns the set of values in items, corresponds to the values of the markers of local_maxima

items_dict = {}
for k in items:
    key = str(k)
    items_dict[key] = {}
    x_coords, y_coords = np.nonzero(segmented == k)
    items_dict[key]['x_coords'] = x_coords
    items_dict[key]['y_coords'] = y_coords
    pixel_values = FO.frames[:,x_coords, y_coords]
    items_dict[key]['pixel_values'] = pixel_values
    items_dict[key]['surface'] = pixel_values.shape[1]
    items_dict[key]['mean'] = np.mean(pixel_values, axis = 1)
    items_dict[key]['std'] = np.std(pixel_values, axis = 1)
    items_dict[key]['remains'] = True




In [10]:
plt.loglog(intense)

NameError: name 'intense' is not defined

In [14]:
file_path = file_folder + "/mean_intense.csv" # pops up a window to select your file
intense = np.genfromtxt(file_path, delimiter=',')
file_path = file_folder + "/mean_fluo.csv" # pops up a window to select your file
mean_fluo = normalize(np.genfromtxt(file_path, delimiter=','))
file_path = file_folder + "/mean_fluo_video.csv" # pops up a window to select your file
mean_fluo_video = normalize(np.genfromtxt(file_path, delimiter=','))
intense =  np.mean(intense.reshape(-1, 6), axis=1)


In [34]:
xx = FO.frames[:,0,0].max()
F = FO.frames/xx

In [35]:
plt.imshow(F[0])

In [18]:
fig = plt.figure(figsize = (10,10))

for k in items:
    if k!= 0:
        curve = items_dict[str(k)]['mean']
        curve = np.mean(curve.reshape(-1, 6), axis=1)
        #plt.scatter(eleme_intensity[20:], curve[19:])
        plt.loglog(intense, curve/intense, '.')
    #if k== 0:
    #    curve = items_dict[str(k)]['mean']
    #    curve = np.mean(FO.frames[:,0,0].reshape(-1, 6), axis=1)
    #    #plt.scatter(eleme_intensity[20:], curve[19:])
    #    plt.loglog(intense, curve, '-')

In [16]:
plt.loglog(intense)

In [114]:
def intensity_fluo(parameters,xdata):
    '''
    Calculate an exponetial decay of the form:
    F = Qmax * exp(- alpha * xdata/Qmax)
    '''
    Qmax = parameters[0]
    alpha = parameters[1]
    return Qmax * (1-np.exp(-alpha * xdata/Qmax))

In [51]:
import numpy as np
from alienlab.regression_func import get_func, regression_affine
from tkinter.filedialog import askopenfilename, askdirectory
import glob
from alienlab.utils import pandas_to_arrays
import pandas as pd
import numpy as np
from NIControl.RoutinesClass import Routines
import sys
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

from config_DAQ import *
from ThorlabsControl.DC4100_LED import ThorlabsDC4100
from ThorlabsControl.FW102 import FW102C
from time import time 

class VoltageIntensity():
    def __init__(self):
        experiment_folder = "G:/DREAM/from_github/PAMFluo/Experiments/2021-06-01_18_10_bode_diagram"
        #"G:\DREAM/from_github\PAMFluo\Experiments/2021-05-19_18_30_bode_diagram"#askdirectory()
        headers, I480 = pandas_to_arrays(glob.glob(experiment_folder + "/*light_intensity_480.csv")[0])
        headers, I405 = pandas_to_arrays(glob.glob(experiment_folder + "/*light_intensity_405.csv")[0])
        self.voltage = {}
        self.voltage['blue'] = I480[1]
        self.voltage['purple'] =  I405[1]
        self.intensity = {}
        self.intensity['blue'] = I480[2]
        self.intensity['purple'] = I405[2]
        self.DO_spectrum = pd.read_csv("G:/DREAM/from_github/PAMFluo/specs/DO_wheel.csv")
        self.detector_response = {}
        self.detector_response["blue"] = pandas_to_arrays(glob.glob(experiment_folder + "/*Detector_response_curve_blue.csv")[0])[1]
        self.detector_response["purple"] = pandas_to_arrays(glob.glob(experiment_folder + "/*Detector_response_curve_purple.csv")[0])[1]



    def get_DO_val(self, LED_color, DO):
        if LED_color == 'blue':
            wlgh = 480
        if LED_color == 'purple':
            wlgh = 405
        if DO != 0:
            func = get_func(self.DO_spectrum['wavelength'], self.DO_spectrum[str(DO)])
            density = func(wlgh)
        else:
            density = 0
        return np.float(10**(-density))


    def get_MPPC_voltage(self, LED_color, DO, voltage_input):
        voltage = self.detector_response[LED_color][1]
        MPPC_voltage = self.detector_response[LED_color][2]
        func = get_func(voltage, MPPC_voltage, k = 1)
        density = self.get_DO_val(LED_color, DO)
        print("density:", density)
        return func(voltage_input)*density

 #   def get_intensity_voltage(self):
 #       
 #       func = get_func(intensity, voltage)
 #       include DO

    def get_intensity_MPPC(self, LED_color, DO, MPPC_input):
        density = self.get_DO_val(LED_color, DO)
        MPPC_voltage = self.voltage[LED_color]
        intensity = self.intensity[LED_color]
        func = get_func(MPPC_voltage, intensity)
        return func(MPPC_input/density)*density

    def get_intensity_voltage(self, LED_color, DO, voltage_input):
        MPPC_input = self.get_MPPC_voltage(LED_color, DO, voltage_input)
        return self.get_intensity_MPPC(LED_color, DO, MPPC_input)


    def assert_calibration(self, logger, filter):
        port_DC4100 = "COM5"
        port_filter_wheel = "COM3"
        ctrlLED = ThorlabsDC4100(logger, port = port_DC4100)
        ctrlLED.initialise_fluo()
        ctrlLED.set_user_limit(LED_blue, 1000)
        ctrlLED.set_user_limit(LED_green, 1000)
        ctrlLED.set_user_limit(LED_purple, 1000)
   


        fwl = FW102C(port=port_filter_wheel)

           

 #       send 3 intensity, given voltage, measure MPPC, check predicted voltage 
        routines = Routines()
        routines.experiment_folder("Check_calibration")
        routines.generator_analog_channels = ["ao0"]
        routines.generator_digital_channels = []
        #480
        offset_min = 0.25
        offset_max =  2
        N_points = 5
        amplitude = 0
        routines.excitation_frequency = 10
        routines.num_period = 10
        routines.points_per_period = 10000
        routines.update_rates()

        fwl.move_to_filter(filters[filter])
        offset_range_480, val_480, fluo_range_480, full_output = routines.detector_response_routine(offset_min,
                                                                                    offset_max, amplitude, N_points, color = 'blue')

        predicted_MPPC = self.get_MPPC_voltage('blue', filter, offset_range_480)
        r2 = r2_score(predicted_MPPC, val_480)
        print(r2)
        plt.figure()
        plt.plot(predicted_MPPC, val_480)
        if abs(r2-1) > 0.2:
            print("YOU NEED TO CALIBRATE THIS SET-UP")
            sys.exit()
        else:
            print("CALIBRATION OK")
        return offset_range_480, val_480, fluo_range_480, full_output





In [52]:


N = 18
filters = [3,2,1]
limits_blue_low = np.linspace(1, 45, N//(2*len(filters)))
limits_blue_high = np.linspace(50, 250, N//(2*len(filters)))
limits_blue = np.concatenate((limits_blue_low, limits_blue_high), axis = 0)

eleme_voltage = []
loc_intensity = []
V = VoltageIntensity()
for f in filters:
    for limit_blue in limits_blue:
        eleme_voltage.extend([V.get_MPPC_voltage('blue', f, limit_blue/100)]*6)
        loc_intensity.extend([V.get_intensity_voltage('blue', f, limit_blue/100)]*6)
        
eleme_intensity = V.get_intensity_MPPC('blue', f, intense)

density: 0.0018626565330991875
density: 0.0018626565330991875
density: 0.0018626565330991875
density: 0.0018626565330991875
density: 0.0018626565330991875
density: 0.0018626565330991875
density: 0.0018626565330991875
density: 0.0018626565330991875
density: 0.0018626565330991875
density: 0.0018626565330991875
density: 0.0018626565330991875
density: 0.0018626565330991875
density: 0.01611856195966334
density: 0.01611856195966334
density: 0.01611856195966334
density: 0.01611856195966334
density: 0.01611856195966334
density: 0.01611856195966334
density: 0.01611856195966334
density: 0.01611856195966334
density: 0.01611856195966334
density: 0.01611856195966334
density: 0.01611856195966334
density: 0.01611856195966334
density: 0.057651104622681304
density: 0.057651104622681304
density: 0.057651104622681304
density: 0.057651104622681304
density: 0.057651104622681304
density: 0.057651104622681304
density: 0.057651104622681304
density: 0.057651104622681304
density: 0.057651104622681304
density: 0

In [35]:
plt.plot(eleme_intensity[1:], loc_intensity[:-1], '.')

In [24]:
fig = plt.figure()
ax = plt.gca()
ax.scatter(intense[1:], eleme_intensity[:-1])
ax.set_yscale('log')
ax.set_xscale('log')

In [25]:
plt.loglog(intense[1:], eleme_intensity[:-1])

In [ ]:
plt.loglog(eleme_intensity, mean_fluo_video[1:])